<a href="https://colab.research.google.com/github/bdbejarc/Instrumentos-Opticos/blob/main/Parcial_Punto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
 # Importar librerías

import pandas as pd
import matplotlib as mlt
import numpy as np
import matplotlib.pyplot as plt
import cv2
import PyQt5 # si se trabaja en local, para que #matplolib qt corra

In [19]:
# Para que las gráficas se impriman no en el notebook, sino en una nueva pestaña
%matplotlib qt

# Crear la Matrix y dar realidad física a la simulación




In [20]:
# Coordenadas de mundo
nm = 1e-9
um = 1e-6
mm = 1e-3
cm = 1e-2



# Longitud de onda
w_length = 633*nm

dx = dy = 3.18*um # los diferenciales (tamaño de pixel) dx = L / N = 6.513 mm / 2048 

# Definamos algunas funciones de interés

In [21]:
# Función para graficar complejos
def fun_ploteo_complejo(mat, indicador, escala, mapa_color = 'gray'):
    """
    mat  es una matriz compleja a plotear
    indicador = I para intensidad, A para amplitud, P para fase
    escala = 1 para nada, 0 para logarítmica
    """

    if(indicador =="I"):
        mat = np.abs(mat)**2
    elif(indicador == "A"):
        mat = np.abs(mat)
    elif(indicador == "P"):
        mat = np.angle(mat)

    if (escala == 1):
        plt.figure()
        plt.imshow(mat, cmap = mapa_color)
        plt.colorbar()
        plt.show()
    else:    
        plt.figure()
        plt.imshow( np.log( mat + 0.000000001), cmap = mapa_color)
        plt.colorbar()
        plt.show()

#----------------------------------------------------------------------------------------------------------------
# Función que convierte campos de 3 canales a un solo canal 
def fun_monocromador(campo_colorado, canal = 0):
    '''
    fun_monocromador es una función que extrae los datos de un solo canal de un campo, se puede entender
    como si un campo se compone de diferentes frecuencias, seleccionar solo los valores de esa frecuencia en 
    en particular que conforman el campo. 
    
    Esta función acepta solo campos con más de un solo canal, si mete de 1 canal, saldrán datos incorrectos

    Variables de entrada
     - campo_colorado: (numpy.ndarray) ; es el campo al cual se filtrará solo los componentes de una frecuencia que
                                         lo consituyen, tiene dimensiones L_x,L_y,canales
     - canal: (int) ; es el índice de los datos correspondientes a la frecuencia que se desea seleccionar  
                      por ejemplo una imagen RGB tiene las frecuencias de rojo en canal = 0, verde en canal = 1,
                      y azul en canal = 2.
                      Por defecto se escoge el canal 0     
                                                  
    '''

    campo_monocromático = campo_colorado[:,:,canal] #Tomar todas las filas y columnas, pero solo los valores del canal
                                                    # de interés

    return (campo_monocromático)

#----------------------------------------------------------------------------------------------------------------
# Función para "Pading"
def fun_pad(campo):
    '''
    fun_pad es una función que "padea" un campo (imagen) 2D con ceros tal que duplica su longitud y altura

    Solo recibe imágenes de 1 canal

    Variables de entrada:
     - campo: (numpy.ndarray) ; es el campo a padear de L_x y L_y dimensiones físicas reales

    Variables de salida:
     - campo_paded: (numpy.ndarray) ; es el campo padeado con ceros de dimensiones 2*L_x y 2*L_y 
    '''
    # extraer las dimensiones del pad
    dimension_pad = np.array( [np.shape(campo)[0] , np.shape(campo)[1] ] ) / 2

    # padear el campo con ceros
    campo_paded = np.pad(campo, ( (int(dimension_pad[0]), int(dimension_pad[0]) ), (int(dimension_pad[1]), int(dimension_pad[1])) ), mode = 'constant', constant_values = (0,0) )

    return (campo_paded)

#----------------------------------------------------------------------------------------------------------------
# Función extractora de características de un campo (imagen) 2D

# Nota: Función por depurar y mejorar,, no está terminada, saltarla
def fun_extractor(campo, L_x, L_y, dx, dy):
    '''
    fun_extractor es una función que extrae las características de interés de un campo (imagen) 2D
    que puede ser de 1 o  3 canales de tamaño con dimensiones físicas reales L_x * L_y que fue medida
    con pixeles de tamaño dx,dy 

    Variables de entrada:
    - campo: (numpy.ndarray) ; Es la imagen que vamos a difractar
    - L_x: (int) ; Dimension x del campo
    - L_y: (int) ; Dimension y del campo
    - dx: (int) ; Dimension x del pixel
    - dy: (int) ; Dimension y del pixel

    Variables de salida:
    - campo_data: (numpy.ndarray) ; Arreglo 2x2, su primer elemento tiene el número de muestras en x y en y
                                  el segundo tiene los deltas en x y en y.

    '''
    # Tamaño de la muestra
    N_x = int(L_x/dx)
    N_y = int(L_y/dy)


    campo_data =np.array( [N_x, N_y] )

    return(campo_data)
# Función para graficar complejos
def fun_ploteo_complejo(mat, indicador, escala, mapa_color = 'gray'):
    '''
    mat  es una matriz compleja a plotear
    indicador = I para intensidad, A para amplitud, P para fase
    escala = 1 para nada, 0 para logarítmica
    '''
    if(indicador =="I"):
        mat = np.abs(mat)**2
    elif(indicador == "A"):
        mat = np.abs(mat)
    elif(indicador == "P"):
        mat = np.angle(mat)

    if (escala == 1):
        plt.figure()
        plt.imshow(mat, cmap = mapa_color)
        plt.colorbar()
        plt.show()
    else:    
        plt.figure()
        plt.imshow( np.log( mat + 0.000000001), cmap = mapa_color)
        plt.colorbar()
        plt.show()

#----------------------------------------------------------------------------------------------------------------
# Función que convierte campos de 3 canales a un solo canal 
def fun_monocromador(campo_colorado, canal = 0):
    '''
    fun_monocromador es una función que extrae los datos de un solo canal de un campo, se puede entender
    como si un campo se compone de diferentes frecuencias, seleccionar solo los valores de esa frecuencia en 
    en particular que conforman el campo. 
    
    Esta función acepta solo campos con más de un solo canal, si mete de 1 canal, saldrán datos incorrectos

    Variables de entrada
     - campo_colorado: (numpy.ndarray) ; es el campo al cual se filtrará solo los componentes de una frecuencia que
                                         lo consituyen, tiene dimensiones L_x,L_y,canales
     - canal: (int) ; es el índice de los datos correspondientes a la frecuencia que se desea seleccionar  
                      por ejemplo una imagen RGB tiene las frecuencias de rojo en canal = 0, verde en canal = 1,
                      y azul en canal = 2.
                      Por defecto se escoge el canal 0     
                                                  
    '''

    campo_monocromático = campo_colorado[:,:,canal] #Tomar todas las filas y columnas, pero solo los valores del canal
                                                    # de interés

    return (campo_monocromático)

#----------------------------------------------------------------------------------------------------------------
# Función para "Pading"
def fun_pad(campo):
    '''
    fun_pad es una función que "padea" un campo (imagen) 2D con ceros tal que duplica su longitud y altura

    Solo recibe imágenes de 1 canal

    Variables de entrada:
     - campo: (numpy.ndarray) ; es el campo a padear de L_x y L_y dimensiones físicas reales

    Variables de salida:
     - campo_paded: (numpy.ndarray) ; es el campo padeado con ceros de dimensiones 2*L_x y 2*L_y 
    '''
    # extraer las dimensiones del pad
    dimension_pad = np.array( [np.shape(campo)[0] , np.shape(campo)[1] ] ) / 2

    # padear el campo con ceros
    campo_paded = np.pad(campo, ( (int(dimension_pad[0]), int(dimension_pad[0]) ), (int(dimension_pad[1]), int(dimension_pad[1])) ), mode = 'constant', constant_values = (0,0) )

    return (campo_paded)

#----------------------------------------------------------------------------------------------------------------
# Función calculadora del z crítico
def fun_z_c(M, dx, w_length):
    '''
    fun_z_c es una función que calcula el la distancia de propagación crítica (z crítico) donde los métodos
    de propagación AS y Fresnel funcionan, dados un campo, tamaño de discretización y longitu de onda
    
    Variables de entrada
     - M: int ; número de muestreos
     - dx: float ; tamaño de discretización (tamaño del pixel)
     - w_lenght: float ; longitud de onda

    Variables de salida
     - z_c: float ; z crítico
    '''
    z_c = M*dx**2/w_length

    return z_c
#----------------------------------------------------------------------------------------------------------------
# Función extractora de características de un campo (imagen) 2D

# Nota: Función por depurar y mejorar,, no está terminada, saltarla
def fun_extractor(campo, L_x, L_y, dx, dy):
    '''
    fun_extractor es una función que extrae las características de interés de un campo (imagen) 2D
    que puede ser de 1 o  3 canales de tamaño con dimensiones físicas reales L_x * L_y que fue medida
    con pixeles de tamaño dx,dy 

    Variables de entrada:
    - campo: (numpy.ndarray) ; Es la imagen que vamos a difractar
    - L_x: (int) ; Dimension x del campo
    - L_y: (int) ; Dimension y del campo
    - dx: (int) ; Dimension x del pixel
    - dy: (int) ; Dimension y del pixel

    Variables de salida:
    - campo_data: (numpy.ndarray) ; Arreglo 2x2, su primer elemento tiene el número de muestras en x y en y
                                  el segundo tiene los deltas en x y en y.

    '''
    # Tamaño de la muestra
    N_x = int(L_x/dx)
    N_y = int(L_y/dy)


    campo_data =np.array( [N_x, N_y] )

    return(campo_data)

#----------------------------------------------------------------------------------------------------------------

   # Función del kernel de una transformada discreta de Fourier y su inversa exp(+-(1j/(2pi))*(n*p/N_x + m*q*N_y))
def fun_fourker( cont_n_m, cont_p_q, N_array, ift = False ):
    '''
    Función  fun_fourker: es una función que calcula la exponencial exp(+-(1j/(2pi*N))*(p*n + q*m)) siendo el signo
    + para la transformada invera y - para la transformada normal
     
    Nota: 1j lo utilizamos para representar el complejo i

    Variables de entrada:
    - cont_n_m: (np.ndarray 1x2) ; arreglo que contiene los contadores n y m del espacio real (x,y)
    - cont_p_q: (np.ndarray 1x2) ; arreglo que contiene los contadores p y q del espacio recíprocro (fx,fy)
    - N_array: (np.ndarray 1x2) ; arreglo que contiene el número de las muestras N_x en x y N_y en y
    - ift: (bool) ; indica si se está calculando transformada o su inversa, por defecto es False
        
    Variables de salida:
    - fourker: (complex) ; valor exponencial evaluada con las variables de entrada

    '''
        #descomprimir
    n = cont_n_m[0]
    m = cont_n_m[1]

    p = cont_p_q[0]
    q = cont_p_q[1]

    N_x = N_array[0]
    N_y = N_array[1]

        
    if(ift == False):
        fourker = np.exp(-(2j*np.pi * (n*p/N_x + m*q/N_y) ) )
    else:
        fourker = np.exp( (2j*np.pi * (n*p/N_x + m*q/N_y) ) )
            
    return fourker
#----------------------------------------------------------------------------------------------------------------
# Función que calcua la transformada discreta de un campo ya sea real o reciproco

def fun_DFT(campo, delta_array, ift = False):
    '''
    fun_DFT es una función que calcula la Transformada de Fourier Discreta (DFT) o inversa (iDFT) de un campo sin 
    tener en cuenta los deltas, por medio de el uso de ciclos for.

    Variables de entrada:
     - campo: (numpy.ndarray) ; campo (arreglo matricial) al cual campo de que se calculará la DFT o iDFT
     - delta_array: (np.ndarray 1x2) ; arreglo que contiene los deltas o tamaños de pixeles dx y dy
     - ift: (bool) ; Si True, calcula la iDFT, si False calcula DFT

    Variables de salida:
     - campo_transformado: (numpy.ndarray) ; matriz de igual dimensiones que campo que contiene los valores de 
                                             la DFT o iDFT 
    '''

    # Crear arreglo lleno de ceros de mismas dimensiones que campo donde guardaré la información de la transformada
    campo_transformado = np.zeros(np.shape(campo), dtype = complex)

    # Dimensiones de campo para contadores
    N_x, N_y = np.shape(campo)

    N_array = np.array([N_x,N_y])

    # Extraer deltas
    dx, dy = delta_array

    L_x = dx*N_x
    L_y = dy*N_y
    


    # variable de escalada si se hace DFT o iDFT
    if(ift == False):
        escalador = dx*dy
    else:
       escalador = 1/(L_x*L_y)

    # definir una variable que alberge el valor de las sumas
    suma = 0

    # DFT
    for p in np.arange(N_x):
        for q in np.arange(N_y):

            for n in np.arange(N_x):
                for m in np.arange(N_y):

                    cont_n_m = np.array([n,m])
                    cont_p_q = np.array([p,q])

                    suma += campo[n,m] * fun_fourker(cont_n_m, cont_p_q, N_array, ift)
 
            
            #Guardar el dato de la transformada para una frecuencia p,q
            campo_transformado[p,q] = suma

            # Reiniciar el sumador para calcular otra frecuencia p,q
            suma = 0
    
    # escalar el campo
    campo_transformado = campo_transformado * escalador

    return campo_transformado

    
#----------------------------------------------------------------------------------------------------------------

def fun_DFT2(campo, delta_array, ift = False):
    '''
    fun_DFT2 es otra función que hace DFT, las variables de entrada y salida son los mismos que fun_DFT
    Esta función calcula por medio de funciones map() nativas de python.
    '''

   # Dimensiones de campo para contadores
    N_x, N_y = np.shape(campo)

    N_array = np.array([N_x,N_y])

    # Extraer deltas
    dx, dy = delta_array

    L_x = dx*N_x
    L_y = dy*N_y

    # variable de escalada si se hace DFT o iDFT
    if(ift == False):
        escalador = dx*dy
    else:
        escalador = 1/(L_x*L_y)

    # Crear una matriz de N_x filas con N_y columnas donde cada elemento de la matriz es un arreglo 1x2 donde el primer
    # valor es el número de la fila y el segundo es el número de la columna

    # ejemplo siendo
    #  _                       _
    # | [0,0]    [0,1]    [0,2] |
    # | [1,0]    [1,1]    [1,2] |
    # | [2,0]    [2,1]    [2,2] |
    #  -                       -
    # la dimension es (N_x, N_y, 2)
    matriz_indexada = np.indices((N_x, N_y, 1))
    matriz_indexada = np.concatenate((matriz_indexada[0], matriz_indexada[1]), axis=2)

    def funcion_g(cont_n_m, cont_p_q, N_array, campo, ift):
        multiplicador = campo[tuple(cont_n_m)]*fun_fourker(cont_n_m, cont_p_q, N_array, ift)
        return multiplicador

    def funcion_f(cont_p_q,  N_array, campo, ift):
        punto_transformado = np.sum( np.array(list(map(lambda filas_n_m: np.array(list(map(lambda n_m: funcion_g(n_m, cont_p_q, N_array, campo, ift ), filas_n_m) ) ) , matriz_indexada ) ) ) )
        return punto_transformado


    matriz_DFT = np.array(list(map(lambda filas_p_q: list(map(lambda p_q: funcion_f(p_q, N_array, campo, ift), filas_p_q) ),matriz_indexada) ) )

    matriz_DFT = matriz_DFT * escalador

    
    return matriz_DFT

#----------------------------------------------------------------------------------------------------------------
def fun_DFT_poderosa(campo, delta_array, ift = False):
    '''
    fun_DFT_poderosa es una función que calcula la Transformada de Fourier Discreta (DFT) o inversa (iDFT) de un campo.

    NOTA 1: Esta función es equivalente a np.fft.fftn(campo*delta_array[0]*delta_array_[1])
    NOTA 2: Esta función aunque bastante óptima para DFT, es muy costosa, puede que su computador colapse, en computador del programador
            se garantiza, corrió una de 128x128, no se probó de mayores tamaños.

    Variables de entrada:
     - campo: (numpy.ndarray) ; campo (arreglo matricial) al cual se calculará la DFT o iDFT
     - delta_array: (np.ndarray 1x2) ; arreglo que contiene los deltas o tamaños de pixeles dx y dy
     - ift: (bool) ; Si True, calcula la iDFT, si False calcula DFT

    Variables de salida:
     - DFT: (numpy.ndarray) ; matriz de igual dimensiones que campo que contiene los valores de 
                              la DFT o iDFT 
    '''

    # Número de muestras del campo
    N,M = np.shape(campo)

    # Extraer deltas
    dx, dy = delta_array

    # Tamaño del campo
    Lx = N*dx
    Ly = M*dy
    
    # Definir variables si se hace DFT o iDFT
    # para iDFT
    if ift == True:
        s = +1                            # signo positivo si iDFT   
        escalador = 1/(Lx*Ly)             # delta espectral
    # para DFT
    else:
        s = -1
        escalador = dx*dy

    # Crear contadores
    n = np.arange(0,N,1)
    m = np.arange(0,M,1)

    p = np.arange(0,N,1)
    q = np.arange(0,M,1)

    # Crear arreglos matriciales
    n_mat, m_mat = np.meshgrid(n,m)
    p_mat, q_mat = np.meshgrid(p,q)

    # Realizar producto tensorial de p x n y q x m, para tener todas las combinaciones
    # posibles de los n,m con un único valor p,q para todos los p y q
    p_tensor_n = np.tensordot(p_mat, n_mat, 0)
    q_tensor_m = np.tensordot(q_mat, m_mat, 0)

    # Dividir para el número de muestras
    p_tensor_n = p_tensor_n/N
    q_tensor_m = q_tensor_m/M

    # kernel de fourier
    fourker = np.exp(1j*2*np.pi*s*(p_tensor_n + q_tensor_m) )

    # multiplicar el campo por el kernel, donde se mantiene la relación punto a punto
    producto = campo * fourker

    # sumar todas las multiplicaciones del campo con el kernel y obtener la matriz p,q
    suma = np.sum(producto, (2,3))

    # Escalar la matriz por los deltas
    DFT = suma * escalador

    return DFT

def fun_metodo_DFT(campo, delta_array, ift, metodo):
    '''
    fun_metodo_DFT es una función que calcula DFT o iDFT por el método de numpy o los programados en este código.

    Variables de entrada
     - campo: (np.ndarray) ; campo al que se le calculará DFT
     - delta_array: (np.ndarray 1x2) ; arreglo que contiene los deltas o tamaños de pixeles dx y dy
     - ift: (bool) ; Si True, calcula la iDFT, si False calcula DFT
     - metodo: (int) ; Puede tomar los valores:
                        0 para calcular DFT con la función de numpy np.fft.fftn
                        1 para calcular DFT con la función fun_DFT
                        2 para calcular DFT con la función fun_DFT2
                        3 para calcular DFT con la función fun_DFT_poderosa
    
    Variables de salida
     - DFT_campo (np.ndarray) ; DFT (o iDFT) del campo de entrada

    '''

    if ift == True:
        escalador = 1/(dx*dy)
    else:
        escalador = dx*dy

    # Por defecto corre la de numpy FFT
    if metodo == 0:    
        DFT_campo = (np.fft.fftn(campo))#*escalador
    # Estos métodos son los programados de DFT, se utilizan solo para comprobar la velocidad de DFT Vs FFT, cada uno es más óptimo 
    # que el anterior, pero a su vez más costoso, probarlo máximo con imágenes de 128x128
    elif metodo == 1:
        DFT_campo = fun_DFT(campo, delta_array, ift )
    elif metodo == 2:
        DFT_campo = fun_DFT2(campo, delta_array, ift )
    elif metodo == 3:
        DFT_campo = fun_DFT_poderosa(campo, delta_array, ift)
    
    return DFT_campo


## Implementación computacional de Difracción por Transformada de Fresnel

In [22]:
def Fresnel (campo, w_length, z, dx, dy, metodo = 0):
    '''
    Fresnel es una función que calcula la propagación por medio de la implementación computacional de transformada de Fresnel

    Variables de entrada
     - campo: campo: (np.ndarray) ; campo al que se le calculará DFT
     - w_length: (float) ; longitud de onda
     - z: (float) ; distancia de propagación
     - dx: (int) ; delta de discretización en x del espacio o largo del pixel
     - dy: (int) ; delta de discretización en y del espacio o alto del pixel
     - metodo: (int) ; Puede tomar los valores:
                        0 para calcular DFT por FFT (o iFFT) con la función de numpy np.fft.fftn (valor por defecto)
                        1 para calcular DFT con la función fun_DFT
                        2 para calcular DFT con la función fun_DFT2
                        3 para calcular DFT con la función fun_DFT_poderosa

    Variables de salida
     - prop: (np.ndarray) ; campo propagado a la distancia z por medio de transformada de Fresnel computacional

    '''
    # Vector de onda
    k_vect = 2*np.pi/w_length


    # Definir espacio coordenado
    N,M = np.shape(campo)
    x = np.arange(-int(M/2), int(M/2), 1)
    y = np.arange(-int(N/2), int(N/2), 1)
    X,Y = np.meshgrid(x,y)

    # Calcular  el espacio de discretización del campo de entrada
    dx_0 = (w_length * z)/(dx * N)
    dy_0 = (w_length * z)/(dy * M) 

    # Definir fases esfericas
    C1 = np.exp( (-1j*k_vect/(2*z) ) * ( (X*dx)**2 + (Y*dy)**2) )
    C2 = np.exp( (1j*k_vect*z) )* np.exp ( (1j*k_vect/(2*z))*( (X*dx_0)**2 + (Y*dy_0)**2) )

    # 2. Preparar U'
    U_p = campo * C1

    # 3. DFT// de U' para obtener U'' en una distancia de propagacion z
    U_2p = np.fft.fftshift(fun_metodo_DFT(U_p, np.array([dx,dy]), False, metodo) )

    # 4. Escalar U
    prop = U_2p * C2

    return prop

In [23]:
holograma = cv2.imread('Hologram.tiff',0)
plt.figure()
plt.imshow(holograma)
plt.show()
np.shape(holograma)

(2048, 2048)

In [24]:
Transformada = np.fft.fftshift(np.fft.fftn(holograma))
fun_ploteo_complejo (Transformada, "I", 0)

In [25]:
Filtro = np.zeros((2048, 2048))
cv2.circle(Filtro, (325,1475), 120, 10, -1)

plt.figure()
plt.imshow(Filtro)
plt.colorbar()
plt.show()

In [26]:
Filtrado = Transformada * Filtro #Eliminación orden cero y orden -1
fun_ploteo_complejo(Filtrado, "I", 0)

**"Onda plana monocromática"**

In [27]:
def Plane_Wave(angleX,angleY, dx, dy, w_length):
  
  k = 2 * np.pi / w_length
  M,N = np.shape(holograma)

  Mcenter = int(M/2)
  Ncenter = int(N/2)

  x = np.arange(-Mcenter+1, Mcenter+1)
  y = np.arange(-Ncenter+1, Ncenter+1)
  
  X, Y = np.meshgrid(x,y)

  X = X * dx
  Y = Y * dy
  
  Ax = np.cos(angleX)
  Ay = np.cos(angleY)

  wave = np.exp(1j * k * (Ax * X + Ay * Y))  #Amplitud = 1

  return wave


In [28]:
Referencia_Conjugada = Plane_Wave(-0.2618, -0.567, dx, dy, w_length)
fun_ploteo_complejo(Referencia_Conjugada, "P", 1)

In [29]:
Transformada_Inversa = np.fft.ifftn(Filtrado)
fun_ploteo_complejo(Transformada_Inversa, "I", 1)

In [30]:
Campo_Complejo = Transformada_Inversa * Referencia_Conjugada

In [32]:
Imagen_Amplitud = Fresnel(Campo_Complejo, w_length, 73*mm, dx, dy, 0)
fun_ploteo_complejo(Imagen_Amplitud, "A", 1)